In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[ 0.0251,  0.0559, -0.0631,  ...,  0.0972,  0.0378,  0.0354],
        [ 0.0720, -0.0330,  0.0335,  ...,  0.0809, -0.0304, -0.0005],
        [-0.0671,  0.0400,  0.0385,  ...,  0.0914,  0.0082,  0.0758],
        ...,
        [ 0.0305,  0.0735,  0.0412,  ...,  0.0411,  0.0482,  0.0262],
        [-0.0362,  0.0759, -0.0943,  ...,  0.0613, -0.0939, -0.0570],
        [-0.0079, -0.0826, -0.0188,  ..., -0.0682, -0.0169, -0.0575]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0355, -0.0249, -0.0187,  0.0687,  0.0335,  0.0260,  0.0341,  0.0950,
         0.0781, -0.0758, -0.0838,  0.0340,  0.0447, -0.0077, -0.0992,  0.0946,
        -0.0007, -0.0339, -0.0785,  0.03

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.1529, 0.6699, 0.8536]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.4029, -0.1462,  0.5251],
        [ 0.1011,  0.3084, -0.2215]], requires_grad=True)
Parameter containing:
tensor([0.3361, 0.2033], requires_grad=True)


Output:
tensor([[0.7480, 0.2362]], grad_fn=<AddmmBackward0>)


In [3]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.4181, 0.4854, 0.5844, 0.7842, 0.0857, 0.9015],
         [0.2042, 0.9203, 0.2875, 0.4005, 0.8723, 0.9065],
         [0.1713, 0.7567, 0.1340, 0.7769, 0.8918, 0.7976],
         [0.8905, 0.3588, 0.8119, 0.2661, 0.5067, 0.3514],
         [0.1560, 0.4164, 0.3195, 0.8881, 0.3984, 0.4312],
         [0.7907, 0.4670, 0.4179, 0.5776, 0.9819, 0.8852]]])
tensor([[[0.9203, 0.9065],
         [0.8905, 0.9819]]])


In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[24.5237, 12.0617, 11.2152,  5.4603],
         [21.5989, 24.5511, 15.7727, 12.9225],
         [10.4666, 23.1640,  6.8282, 23.9101],
         [15.6578,  7.9309, 22.1772, 24.9522]]])
tensor(16.4496)
tensor([[[ 1.6123, -0.1803, -0.3021, -1.1299],
         [ 0.6279,  1.2699, -0.6390, -1.2588],
         [-0.7442,  0.9355, -1.2254,  1.0341],
         [-0.3081, -1.4856,  0.6854,  1.1083]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(5.0291e-08, grad_fn=<MeanBackward0>)


In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.3510, 1.2122, 1.6399, 0.1124],
         [0.0000, 0.5110, 0.0000, 0.0000],
         [0.2476, 0.0000, 1.6213, 0.0000],
         [1.5039, 0.8329, 0.0000, 1.1681]]])
tensor([[[0.3510, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.5110, 0.6878, 1.5549],
         [0.2476, 0.0000, 0.0000, 0.0000],
         [1.5039, 0.0000, 1.1585, 1.1681]]])
